In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import time
import os

In [3]:
myfile=[]
for file in os.listdir("../data/"):
    myfile.append("../data/"+file)

In [4]:
myfile

['../data/airpollution-2018-05-09-12-38.json',
 '../data/airpollution-2018-05-10-09-23.json',
 '../data/airpollution-2018-05-11-09-20.json',
 '../data/airpollution-2018-05-11-22-55.json',
 '../data/airpollution-2018-05-12-12-35.json',
 '../data/airpollution-2018-05-13-12-27.json',
 '../data/airpollution-2018-05-14-10-47.json',
 '../data/airpollution-2018-05-15-00-45.json',
 '../data/airpollution-2018-05-15-11-17.json',
 '../data/airpollution-2018-05-16-11-22.json',
 '../data/airpollution-2018-05-16-15-08.json',
 '../data/airpollution-2018-05-17-09-40.json',
 '../data/airpollution-2018-05-18-01-30.json',
 '../data/airpollution-2018-05-19-18-07.json']

In [5]:
tweets_data=[]
for thisfile in myfile:
    with open(thisfile, 'r') as f:
        for line in f:
            try:
                tweet=json.loads(line)
                tweets_data.append(tweet)
            except:
                continue

In [6]:
tweets_data[2775]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu May 10 12:00:57 +0000 2018',
 'display_text_range': [0, 140],
 'entities': {'hashtags': [{'indices': [21, 34], 'text': 'AirPollution'}],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/9…',
    'expanded_url': 'https://twitter.com/i/web/status/994547840276983808',
    'indices': [117, 140],
    'url': 'https://t.co/6SfuzxBbp6'}],
  'user_mentions': [{'id': 14499829,
    'id_str': '14499829',
    'indices': [63, 67],
    'name': 'WHO',
    'screen_name': 'WHO'},
   {'id': 601922436,
    'id_str': '601922436',
    'indices': [81, 94],
    'name': 'Dr Maria Neira',
    'screen_name': 'DrMariaNeira'}]},
 'extended_tweet': {'display_text_range': [0, 149],
  'entities': {'hashtags': [{'indices': [21, 34], 'text': 'AirPollution'},
    {'indices': [132, 148], 'text': 'CleanAir4Health'}],
   'symbols': [],
   'urls': [{'display_url': 'twitter.com/drmarianeira/s…',
     'expanded_url': 'https://twitter.com/drm

In [7]:
tweets=pd.DataFrame()# store the final result

In [8]:
# text: the text of the tweet itself
# created_at: the date of creation
# favorite_count, retweet_count: the number of favourites and retweets
# favorited, retweeted: boolean stating whether the authenticated user (you) have favourited or retweeted this tweet
# lang: acronym for the language (e.g. “en” for english)
# id: the tweet identifier
# place, coordinates, geo: geo-location information if available
# user: the author’s full profile
# entities: list of entities like URLs, @-mentions, hashtags and symbols
# in_reply_to_user_id: user identifier if the tweet is a reply to a specific user
# in_reply_to_status_id: status identifier id the tweet is a reply to a specific status

In [9]:
def get_retweetid(tweet):
    try:
        return(tweet["retweeted_status"]["id"])
    except: 
        return None
    
def get_retweet_username(tweet):
    try:
        return(tweet["retweeted_status"]["user"]["name"])
    except: 
        return None

def get_retweet_followers_count(tweet):
    try:
        return(int(tweet["retweeted_status"]["user"]["followers_count"]))
    except: 
        return None
    
def get_retweet_user_id(tweet):
    try:
        return(tweet["retweeted_status"]["user"]["id_str"])
    except: 
        return None
    
def get_retweet_fa(tweet):
    try:
        return(int(tweet["retweeted_status"]["favorite_count"]))
    except: 
        return None
def get_retweet_retw(tweet):
    try:
        return(int(tweet["retweeted_status"]["retweet_count"]))
    except: 
        return None
def get_retweet_rep(tweet):
    try:
        return(int(tweet["retweeted_status"]["reply_count"]))
    except: 
        return None
    
def get_quoted_status_text(tweet):
     try:
        return(tweet["quoted_status"]["extended_tweet"]["full_text"])
     except:
        return None

In [10]:
tweets=pd.DataFrame()
tweets['id'] = list(map(lambda tweet: int(tweet['id']), tweets_data))
# tweets['country']=list(map(lambda tweet: tweet['place']['country'] if tweet['place']!= None else "", tweets_data))
tweets['text']=list(map(lambda tweet: tweet['text'], tweets_data))
tweets["user_id"]=list(map(lambda tweet:tweet["user"]["id_str"],tweets_data))
tweets["user_name"]=list(map(lambda tweet:tweet["user"]["name"],tweets_data))
tweets["user_followers"]=list(map(lambda tweet:int(tweet["user"]["followers_count"]),tweets_data))

# tweets["quoted_status_text"]=list(map(lambda tweet:int(tweet["quoted_status"]["extended_tweet"]["full_text"]),tweets_data))
tweets["quoted_status_text"]=list(map(lambda tweet:get_quoted_status_text(tweet),tweets_data))


tweets['user_loc']=list(map(lambda tweet: tweet['user']['location'], tweets_data))
tweets['lang']=list(map(lambda tweet: tweet['lang'], tweets_data))
# tweets['retweeted'] = list(map(lambda tweet: tweet['retweeted'], tweets_data))
tweets['retweet_id']=list(map(get_retweetid, tweets_data))

tweets['retweet_user_id']=list(map(get_retweet_user_id, tweets_data))

tweets['retweet_status_user_name']=list(map(get_retweet_username, tweets_data))
tweets['retweet_status_follower_count']=list(map(get_retweet_followers_count, tweets_data))


tweets['retweet_status_favorite_count']=list(map(get_retweetid, tweets_data))
tweets['retweet_status_retweet_count']=list(map(get_retweet_retw, tweets_data))
tweets['retweet_status_reply_count']=list(map(get_retweet_rep, tweets_data))

tweets['created_at']=list(map(lambda tweet: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')), tweets_data))


# tweets['retweet_count']=list(map(lambda tweet: tweet['retweet_count'] if tweet['retweet_count']!= None else None, tweets_data))
# tweets['reply_count']=list(map(lambda tweet: tweet['reply_count'] if tweet['reply_count']!= None else None, tweets_data))
# tweets['favorite_count']=list(map(lambda tweet: tweet['favorite_count'], tweets_data))


In [11]:
# get rid of replicated id
tweets=tweets.drop_duplicates()

In [12]:
tweets=tweets.loc[tweets.lang=="en",] # getrid of none enligsh

In [13]:
tweets.shape

(26777, 16)

In [14]:
sum(tweets.quoted_status_text.isna())

24088

# tokenization and stop words


In [16]:
from nltk.tokenize import word_tokenize
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
#     print(tokens_re.search(tokens[0]))
    if lowercase:
        tokens =  [token if emoticon_re.search(token)  else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'a n', 'example', '!', ':D', 'http://example.com', '#NLP']


['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']


In [17]:
from nltk.corpus import stopwords
import string
def remove_stop_words(tweet):
    out=tweet
    if(tweet):
        p=re.compile(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs

        punctuation = list(string.punctuation)
        stop = stopwords.words('english') + punctuation + ['via',"\r",r'<[^>]+>']
    #     print(stop)
        terms_stop = [term for term in preprocess(tweet) if p.match(term)==None and term not in stop]
    #     terms_stop = [term for term in preprocess(tweet) if term not in stop]
        out=" ".join(terms_stop)
    return(out)

In [18]:
mytext=list(map(lambda tweet: remove_stop_words(tweet), tweets.text))

In [19]:
tweets["text"]=mytext

In [20]:
user_loc=list(map(lambda tweet: remove_stop_words(tweet), tweets.user_loc))

In [21]:
tweets.user_loc=user_loc

In [22]:
tweets=tweets.loc[tweets.text!="",]

DataFrame.to_csv(path_or_buf=None, sep=', ', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, escapechar=None, decimal='.')[source]¶

In [23]:
tweets.to_csv("../output/clean_data_5_20.csv",encoding="utf-8")

In [282]:
t=pd.read_csv("../output/clean_data_4_20.csv",encoding="utf-8")

In [48]:
t.loc[t.user_name=="Donald J. Trump",]

NameError: name 't' is not defined